In [ ]:
# Import dependencies
import pandas as pd
import requests
import gmaps

# Import the API key
from config import g_key

In [3]:
# Store cities.csv in a DataFrame
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Georgetown,MY,2020-08-20 20:35:47,5.41,100.34,82.40,88,20,3.36
1,1,Coquimbo,CL,2020-08-20 20:33:02,-29.95,-71.34,55.40,76,20,10.29
2,2,Nyurba,RU,2020-08-20 20:35:48,63.28,118.33,39.43,79,3,1.39
3,3,Cherskiy,RU,2020-08-20 20:35:48,68.75,161.30,40.39,70,9,8.08
4,4,Norman Wells,CA,2020-08-20 20:35:49,65.28,-126.83,62.60,48,40,19.46


In [4]:
# Check the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [6]:
# Heatmap of temperature
locations = city_data_df[['Lat', 'Lng']]
max_temp = city_data_df['Max Temp']

# Assign the figure and heatmap variables
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, 
                                 max_intensity=300, point_radius=4)

#Add the heatmap layer
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
# Set maximum temps less than zero to zero
max_temp = city_data_df['Max Temp']
temps = [max(temp, 0) for temp in max_temp]

In [8]:
# Heatmap of percent humidity
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']

# Assign the figure and heatmap variables
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=300, point_radius=4)

#Add the heatmap layer
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[['Lat', 'Lng']]
clouds = city_data_df['Cloudiness']

# Assign the figure and heatmap variables
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False,
                                max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of wind speed
locations = city_data_df[['Lat', 'Lng']]
wind = city_data_df['Wind Speed']

# Assign the figure and heatmap variables
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False,
                                max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer for minimum and maximum temperatures
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities with matching criteria
preferred_cities_df = city_data_df[(city_data_df['Max Temp'] <= max_temp) & 
                                   (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Georgetown,MY,2020-08-20 20:35:47,5.41,100.34,82.40,88,20,3.36
9,9,Phangnga,TH,2020-08-20 20:30:58,8.45,98.53,77.00,94,20,2.24
10,10,Excelsior Springs,US,2020-08-20 20:35:51,39.34,-94.23,82.99,44,1,5.82
11,11,Albany,US,2020-08-20 20:35:52,42.60,-73.97,78.01,30,12,2.57
12,12,Ponta Do Sol,PT,2020-08-20 20:35:52,32.67,-17.10,77.00,73,20,3.36


In [13]:
# Check preferred_cities_df for null values
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [52]:
# Create a DataFrame to store hotel names, city, country, max temp, and coordinates
hotel_df = preferred_cities_df.loc[:,['City', 'Country', 'Max Temp', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Georgetown,MY,82.40,5.41,100.34,
9,Phangnga,TH,77.00,8.45,98.53,
10,Excelsior Springs,US,82.99,39.34,-94.23,
11,Albany,US,78.01,42.60,-73.97,
12,Ponta Do Sol,PT,77.00,32.67,-17.10,


In [53]:
# Set parameters to search for a hotel
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}


In [56]:
# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # Add the latitude and longitude to the params dictionary
    params['location'] = f'{lat},{lng}'
    
    # Create the base URL
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    
    # Make a request and get the JSON data from the results
    hotels = requests.get(base_url, params=params).json()
    
    # Store the name of the first hotel from the results 
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
    
hotel_df.head()


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Georgetown,MY,82.40,5.41,100.34,Cititel Penang
9,Phangnga,TH,77.00,8.45,98.53,Rattanapong Hotel
10,Excelsior Springs,US,82.99,39.34,-94.23,The Elms Hotel & Spa
11,Albany,US,78.01,42.60,-73.97,
12,Ponta Do Sol,PT,77.00,32.67,-17.10,Hotel do Campo


In [60]:
# Add a heatmap of temperature for the vacation spots
locations = hotel_df[['Lat', 'Lng']]
max_temp= hotel_df['Max Temp']

# Assign the figure, heatmap, and marker variables
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                 max_intensity=300, point_radius=4)
markers = gmaps.marker_layer(locations)

# Add the heatmap and marker layers
fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [64]:
# Create the info box content
info_box_template = '''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>
'''
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [66]:
# Add a heatmap of temperature for the vacation spots
locations = hotel_df[['Lat', 'Lng']]
max_temp= hotel_df['Max Temp']

# Assign the figure, heatmap, and marker variables
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                 max_intensity=300, point_radius=4)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the heatmap and marker layers
fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))